# Homework 5

Covers Lectures 17, 21.

## Problem 1: Calibrating the reaction rates $\kappa$ [0-4] in Catalytic Conversion of Nitrate to Nitrogen

This is Example 3.1 of [(Tsilifis, 2014)](http://arxiv.org/abs/1410.5522).

Consider the catalytic
conversion of nitrate ($\mbox{NO}_3^-$) to nitrogen ($\mbox{N}_2$) and other
by-products by electrochemical means.
The mechanism that is followed is complex and not well understood.
The experiment of \cite{katsounaros} confirmed the
production of nitrogen ($\mbox{N}_2$), ammonia
($\mbox{NH}_3$), and nitrous oxide ($\mbox{N}_2\mbox{O}$) as final products
of the reaction, as well as the intermediate production of nitrite ($\mbox{NO}_2^-$).
The data are reproduced in [Comma-separated values](https://en.wikipedia.org/wiki/Comma-separated_values) (CSV) and stored in
[data/catalysis.csv](data/catalysis.csv).
The time is measured in minutes and the conentrations are measured in $\mbox{mmol}\cdot\mbox{L}^{-1}$.
Let's load the data into this notebook using the [Pandas](http://pandas.pydata.org) Python module:


In [1]:
import pandas as pd
catalysis_data = pd.read_csv('../data/catalysis.csv', index_col=0)
catalysis_data

,NO3,NO2,N2,NH3,N2O
Time,,,,,
0,500.00,0.00,0.00,0.00,0.00
30,250.95,107.32,18.51,3.33,4.98
60,123.66,132.33,74.85,7.34,20.14
90,84.47,98.81,166.19,13.14,42.10
120,30.24,38.74,249.78,19.54,55.98
150,27.94,10.42,292.32,24.07,60.65
180,13.54,6.11,309.50,27.26,62.54


The theory of catalytic reactions guarantees that the total mass must be conserved.
However, this is not the case in our dataset:

In [2]:
catalysis_data.sum(axis=1)

Time
0      500.00
30     385.09
60     358.32
90     404.71
120    394.28
150    415.40
180    418.95
dtype: float64

This inconsistency suggests the existence of an intermediate unobserved reaction product X.
[(Katsounaros, 2012)](http://www.sciencedirect.com/science/article/pii/S0013468612005208) suggested that the following reaction path shown in the following figure.

![](../lectures/figures/scheme.png "Reaction Scheme")

The dynamical system associated with the reaction is:
$$
\begin{array}{cc}
\frac{d \left[\mbox{NO}_3^-\right]}{dt} &= -k_1\left[\mbox{NO}_3^-\right], \\
\frac{d\left[\mbox{NO}_2^-\right]}{dt} &= k_1\left[\mbox{NO}_3^-\right] - (k_2 + k_4 +
k_5)[\mbox{NO}_2^-], \\
\frac{d \left[\mbox{X}\right]}{dt} &= k_2 \left[\mbox{NO}_2^-\right] - k_3 [X],\\
\frac{d \left[\mbox{N}_2\right]}{dt} &= k_3 \left[\mbox{X}\right], \\
\frac{d \left[\mbox{NH}_3\right]}{dt} &= k_4 \left[\mbox{NO}_2^-\right],\\
\frac{d \left[\mbox{N}_2O\right]}{dt} &= k_5 \left[\mbox{NO}_2^-\right],
\end{array}
$$
where $[\cdot]$ denotes the concentration of a quantity, and
$k_i > 0$, $i=1,...5$ are the *kinetic rate constants*.


### Questions - Calibrate the reaction rates $\kappa=(\kappa_1,\kappa_2,\kappa_3,\kappa_4,\kappa_5)$ in the Catalysis Model to the Experimental Data

Assume that you are a chemical engineer and that you are assigned the task of designing a reactor for the conversion of nitrate to nitrogen. Before you start designing, you are asked to calibrate the reaction rates $\kappa_1,\kappa_2,\kappa_3,\kappa_4,\kappa_5$ using the observed concentration $y=(y_1,...,y_5)$ at $t=180$ stored in [data/catalysis.csv](data/catalysis.csv).

We wish to calibrate the reaction rate parameters of the model to the data. Because the parameters are two small, it is desired to work with the transformed version:

$$
\xi_i = \log\left(180k_i\right).
$$

You are requested to address this problem by using the following two appeoaches:

#### Approach (1) Use Bayesian inversion method to inverse the reaction rates $\kappa=(\kappa_1,\kappa_2,\kappa_3,\kappa_4,\kappa_5)$ using Bayesian inference and MCMC.

We assume that the observed consentration value $y=(y_1,...,y_5)$ is contaminated with (additive) noise $\epsilon_y$ with unknown scale $\sigma_y^2$ , in log scale; i.e.

$$ \log(y) = \log(u(\xi)) + \epsilon, \  \epsilon \sim \text{N}(0,\sigma_y^2), $$ 

where $u(\xi)$ is the free of errors consentration function with respect to the reaction rates $\kappa=(\kappa_1,\kappa_2,\kappa_3,\kappa_4,\kappa_5)$, but free of errors.

For a given value of $\xi=(\xi_1, \xi_2, \xi_3 \xi_4, \xi_5)$, the consentration function $u(\xi)$ can be computed by running the Computer Model described; see Section `Computer Model' for details.

Therefore, 

##### Statistical model   $\mathcal{L}(y|\xi,\sigma_y^2)$

The likelihood function is:

$$
\mathcal{L}(y|\xi,\sigma_y^2) = \prod_{i=1}^{n=5}\text{LN}(y_i|\log(u(\xi_i)),\sigma_y^2)\ 
$$

where $\text{LN}(\mu,\sigma^2)$ denotes the the Log-Normal distribution with location paremeter $\mu$, and scale parameter $\sigma^2$. 

(Note: $X\sim \text{LN}(\mu,\sigma^2)$ iff $\log(X)\sim \text{N}(\mu,\sigma^2)$).

##### Prior model   $\pi(\xi_1, \xi_2, \xi_3 \xi_4, \xi_5, \sigma^2)$

To account for uncertainty about the unknown reaction rates $\xi$, we assign priors:

$$
\xi_1 \sim \text{N}(\mu_1=1.35, \sigma_1^2 = 0.05^2), \   \xi_1 \in (-\infty, +\infty)
$$
$$
\xi_2 \sim \text{N}(\mu_2=1.65, \sigma_2^2 = 0.08^2), \   \xi_2 \in (-\infty, +\infty)
$$
$$
\xi_3 \sim \text{N}(\mu_3=1.34, \sigma_3^2 = 0.11^2), \   \xi_3 \in (-\infty, +\infty)
$$
$$
\xi_4 \sim \text{N}(\mu_4=-0.16, \sigma_4^2 = 0.16^2), \   \xi_4 \in (-\infty, +\infty)
$$
$$
\xi_5 \sim \text{N}(\mu_5=-3.84, \sigma_5^2 = 0.20^2), \   \xi_5 \in (-\infty, +\infty)
$$

To account for uncertainty about the unknown scale of the noise term, we assign prior:

$$
\sigma_y^2 \sim \text{IG}(a_{\sigma_y^2}=0.001, b_{\sigma_y^2}=0.001), \   \sigma_y^2>0
$$
with density
$$
\text{IG}(x|\alpha, \beta) = 
\begin{cases}
\frac{\beta^\alpha}{\Gamma(\alpha)} x^{-\alpha - 1} \exp \left(\frac{-\beta}{x}\right) & \text{, if } x > 0 \\
0 & \text{, otherwise}
\end{cases}
$$



##### The Bayesian model (synopsis)

$$
\begin{array}{rl}
y \sim & \text{LN}(y_i|\log(u(\xi_i)),\sigma_y^2) \  i=1,...,5 \\
\xi_1 \sim & \text{N}(\mu_1=1.35, \sigma_1^2 = 0.05^2), \\
\xi_2 \sim & \text{N}(\mu_2=1.65, \sigma_2^2 = 0.08^2), \\
\xi_3 \sim & \text{N}(\mu_3=1.34, \sigma_3^2 = 0.11^2), \\
\xi_4 \sim & \text{N}(\mu_4=-0.16, \sigma_4^2 = 0.16^2), \\
\xi_5 \sim & \text{N}(\mu_5=-3.84, \sigma_5^2 = 0.20^2), \\
\sigma_y^2 \sim & \text{IG}(a_{\sigma_y^2}=0.001, b_{\sigma_y^2}=0.001),
\end{array}
$$

##### Posterior model $\pi(\xi_1, \xi_2, \xi_3, \xi_4, \xi_5, \sigma^2|y)$

The posterior distribution is given via the Bayes theorem as:

$$
\begin{array}{rl}
\pi(\xi_1,\xi_2,\xi_3,\xi_4,\xi_5,\sigma_y^2|y) \propto& \mathcal{L}(y|\xi,\sigma_y^2) \pi(\xi_1, \xi_2, \xi_3 \xi_4, \xi_5, \sigma^2) \\
\ \propto & ...
\end{array}
$$

##### Your tasks:

1. Calculate the posterior distribution density up to an unknown normalising constant, a.k.a $\pi(\xi_1,\xi_2,\xi_3,\xi_4,\xi_5,\sigma_y^2|y) \propto ...$

3. Design a Markov chain Monte Carlo sampler (in python) in order to sample from the posterior distribution $\pi(\xi_1, \xi_2, \xi_3 \xi_4, \xi_5, \sigma^2|y)$. 
    1. Why did you design your MCMC sampler like that?
        + Hint: Most of the MCMC schemes and Metropolis-Hastings algorithms presented in the lexture can be used, however some of them may be more preferable)

4. Draw a sample from the posterior $\pi(\xi_1,\xi_2,\xi_3,\xi_4,\xi_5,\sigma_y^2|y)$ via MCMC:
    1. Generate a sample for $\xi_1, \xi_2, \xi_3 \xi_4, \xi_5, \sigma^2$ by simulating the MCMC sampler
    2. Draw the trace plot of the generated MCMC sample for each indivudual variable $\xi_1, \xi_2, \xi_3 \xi_4, \xi_5, \sigma^2$
    3. Report:
        1. the scale parameter of the proposals if Random Walk Matropolis was used
        2. the average acceptance probebilities, if any kind of Metropolis-Hastings alg. were used
        3. the integrated autocorrelation times of the generated sample for each indivudual variable $\xi_1, \xi_2, \xi_3 \xi_4, \xi_5, \sigma^2$
            + Hint: for example, use the function iat() (in python) provided below
    3. Draw the autocorrelation plots of the generated sample for each indivudual variable $\xi_1, \xi_2, \xi_3 \xi_4, \xi_5, \sigma^2$. 
        + Hint: for example, use the function myacorr() (in python) provided below
    4. Write down a short discussion and comments 
        + Does your MCMC sampler works satisfactory? Why ?

4. Perform Bayesian inference on the unknown parameters $\kappa_1,\kappa_2,\kappa_3,\kappa_4,\kappa_5, \sigma^2$, by using the generated MCMC sample.         
    (Hint: A transformation is required; recall that $\xi_i=\log(180\kappa_i)$, $i=1,...,5$, so $\kappa_i=...$)
    1. Compute the posterior averages, (point estimates), of $\kappa_1,\kappa_2,\kappa_3,\kappa_4,\kappa_5, \sigma^2$, as well as the associated standard errors (s.e.).  
        + Hint: the MCMC sample is not independent
    2. Plot the estimated marginal posterior densities of the unknown parameters $\kappa_1,\kappa_2,\kappa_3,\kappa_4,\kappa_5$. 
        + Hint, histogram is accepted as an estimator.
    3. Plot 2D scatter-plots for every pair of the unknown parameters $\kappa_1,\kappa_2,\kappa_3,\kappa_4,\kappa_5$. 
        + Hint: You can use ( pandas.tools.plotting import scatter_matrix )
    4. Write down a short discussion and comments 
        + What conclusions can you draw from the posterior densities, scatter-plots, estimates, s.e. ??? 

5. Use Bayesian global optimization method to inverse the reaction rates $\kappa=(\kappa_1,\kappa_2,\kappa_3,\kappa_4,\kappa_5)$.
Hint: use Bayesian global optimization toolbox - http://sheffieldml.github.io/GPyOpt/. Define and minimize an appropriate loss function, e.g., the L2-error between observations and model predictions.

### Computer Model

This section describes how to compute the function $u(\xi)$, where $\xi=(\xi_1,...,\xi_5)$, $\xi_i=\log(180\kappa_i)$ for $i=1,...,5$, which is required for the computation of the likelihood function.

We will develop a generic computational model for the solution of dynamical systems and we will use it to study the catalysis problem. The code relies on the [Fourth-order Runge-Kutta method](https://en.wikipedia.org/wiki/Runge–Kutta_methods) and is a modified copy of [http://www.math-cs.gordon.edu/courses/ma342/python/diffeq.py](http://www.math-cs.gordon.edu/courses/ma342/python/diffeq.py) developed by Jonathan Senning. The code solves:

$$
\begin{array}{ccc}
\dot{\mathbf{y}} &=& f(\mathbf{y}, t),\\
\mathbf{y}(0) &=& \mathbf{y}_0.
\end{array}
$$


In [3]:
import numpy as np
def rk45( f, y0, t, args=() ):
    """Fourth-order Runge-Kutta method with error estimate.

    USAGE:
        y = rk45(f, x0, t, args=())

    INPUT:
        f     - function of x and t equal to dx/dt.  x may be multivalued,
                in which case it should a list or a NumPy array.  In this
                case f must return a NumPy array with the same dimension
                as x.
        y0    - the initial condition(s).  Specifies the value of x when
                t = t[0].  Can be either a scalar or a list or NumPy array
                if a system of equations is being solved.
        t     - list or NumPy array of t values to compute solution at.
                t[0] is the the initial condition point, and the difference
                h=t[i+1]-t[i] determines the step size h.
        args  - any other parameters of the function f.

    OUTPUT:
        y     - NumPy array containing solution values corresponding to each
                entry in t array.  If a system is being solved, x will be
                an array of arrays.

    NOTES:
        This version is based on the algorithm presented in "Numerical
        Mathematics and Computing" 6th Edition, by Cheney and Kincaid,
        Brooks-Cole, 2008.
    """

    # Coefficients used to compute the independent variable argument of f

    c20  =   2.500000000000000e-01  #  1/4
    c30  =   3.750000000000000e-01  #  3/8
    c40  =   9.230769230769231e-01  #  12/13
    c50  =   1.000000000000000e+00  #  1
    c60  =   5.000000000000000e-01  #  1/2

    # Coefficients used to compute the dependent variable argument of f

    c21 =   2.500000000000000e-01  #  1/4
    c31 =   9.375000000000000e-02  #  3/32
    c32 =   2.812500000000000e-01  #  9/32
    c41 =   8.793809740555303e-01  #  1932/2197
    c42 =  -3.277196176604461e+00  # -7200/2197
    c43 =   3.320892125625853e+00  #  7296/2197
    c51 =   2.032407407407407e+00  #  439/216
    c52 =  -8.000000000000000e+00  # -8
    c53 =   7.173489278752436e+00  #  3680/513
    c54 =  -2.058966861598441e-01  # -845/4104
    c61 =  -2.962962962962963e-01  # -8/27
    c62 =   2.000000000000000e+00  #  2
    c63 =  -1.381676413255361e+00  # -3544/2565
    c64 =   4.529727095516569e-01  #  1859/4104
    c65 =  -2.750000000000000e-01  # -11/40

    # Coefficients used to compute 4th order RK estimate

    a1  =   1.157407407407407e-01  #  25/216
    a2  =   0.000000000000000e-00  #  0
    a3  =   5.489278752436647e-01  #  1408/2565
    a4  =   5.353313840155945e-01  #  2197/4104
    a5  =  -2.000000000000000e-01  # -1/5

    b1  =   1.185185185185185e-01  #  16.0/135.0
    b2  =   0.000000000000000e-00  #  0
    b3  =   5.189863547758284e-01  #  6656.0/12825.0
    b4  =   5.061314903420167e-01  #  28561.0/56430.0
    b5  =  -1.800000000000000e-01  # -9.0/50.0
    b6  =   3.636363636363636e-02  #  2.0/55.0

    n = len( t )
    y = np.array( [ y0 ] * n )
    for i in xrange( n - 1 ):
        h = t[i+1] - t[i]
        k1 = h * f( y[i], t[i], *args )
        k2 = h * f( y[i] + c21 * k1, t[i] + c20 * h, *args )
        k3 = h * f( y[i] + c31 * k1 + c32 * k2, t[i] + c30 * h, *args )
        # BUG: The ``-`` in the equation below should be a ``+``.
        k4 = h * f( y[i] + c41 * k1 + c42 * k2 + c43 * k3, t[i] + c40 * h, *args )
        k5 = h * f( y[i] + c51 * k1 + c52 * k2 + c53 * k3 + c54 * k4, \
                        t[i] + h, *args )
        k6 = h * f( \
            y[i] + c61 * k1 + c62 * k2 + c63 * k3 + c64 * k4 + c65 * k5, \
            t[i] + c60 * h, *args )

        y[i+1] = y[i] + a1 * k1 + a3 * k3 + a4 * k4 + a5 * k5
        y5 = y[i] + b1 * k1 + b3 * k3 + b4 * k4 + b5 * k5 + b6 * k6

    return y

Following we will develop a solver for the catalysis model. All, we need to do is define the right hand side of Eq. (\ref{eq:kinetic_model}):

In [4]:
def f_catalysis(y, t, kappa):
    rhs = np.zeros((6,))
    rhs[0] = -kappa[0] * y[0]
    rhs[1] = kappa[0] * y[0] - (kappa[1] + kappa[3] + kappa[4]) * y[1]
    rhs[2] = kappa[1] * y[1] - kappa[2] * y[2]
    rhs[3] = kappa[2] * y[2]
    rhs[4] = kappa[3] * y[1]
    rhs[5] = kappa[4] * y[1]
    return rhs

Let's try to calibrate the reaction rate parameters of the model to the data. Because the parameters are two small, let us work with the transformed version:

$$
\xi_i = \log\left(180k_i\right).
$$

Therefore, the routine that returns $u(\xi)$ for given values of $\xi_1$, $\xi_2$, $\xi_3$, $\xi_4$, $\xi_5$ is as follows:

In [5]:
def comp_model(xi1, xi2, xi3, xi4, xi5):
    t = np.linspace(0, 180, 100)
    kappa = np.exp([xi1, xi2, xi3, xi4, xi5]) / 180.
    y = rk45(f_catalysis, (500., 0., 0., 0., 0., 0.), t, args=(kappa,))
    return np.array([y[-1,0],y[-1,1],y[-1,3],y[-1,4],y[-1,5]])

## Auxiliary routines for the computation of the integrated autocorrelation function

In [6]:
# ===============================================================
# AUXILIARY FUNCTIONS FOR THE COMPUTATION OF HE AUTOCORRELATIONS
# ===============================================================

import numpy as np
from scipy.linalg import toeplitz
# from __future__ import division, absolute_import, print_function

def myacorr(x) :
    mean_x = x.mean()
    z = np.fft.fft( x-mean_x, int(2**np.ceil( np.log2(2*x.size) ))+1 )
    z = z.real**2 + z.imag**2
    z = np.fft.ifft( z ).real
    z = z[:x.size]
    z = z/z[0]
    return z

# from __future__ import division, absolute_import, print_function
from scipy.linalg import toeplitz

def iat(x):
    r"""Estimate the integrated autocorrelation time, :math:`\tau_{int}` of a
    time series.

    This method estimates the spectral density at zero frequency by fitting
    an AR(p) model, with p selected by AIC.

    Parameters
    ----------
    x : ndarray, shape=(n_samples, n_dims)
        The time series, with time along axis 0.

    References
    ----------
    .. [1] Plummer, M., Best, N., Cowles, K., and Vines, K. (2006). CODA:
        Convergence diagnosis and output analysis for MCMC. R News, 6(1):7-11.

    Returns
    -------
    tau_int : ndarray, shape=(n_dims,)
        The estimated integrated autocorrelation time of each dimension in
        ``x``, considered independently.
    """
    if x.ndim == 1:
        x = x.reshape(-1, 1)
    process_var = np.var(x, axis=0, ddof=1)

    tau_int = np.zeros(x.shape[1])
    for j in range(x.shape[1]):
        # fit an AR(p) model, with p selected by AIC
        rho, sigma2 = yule_walker(x[:,j], order_max=10)
        # power spectral density at zero frequency
        spec0 = sigma2 / (1 - np.sum(rho))**2
        # divide by the variance
        tau_int[j] = spec0 / process_var[j]

    return tau_int

def yule_walker(X, aic=True, order_max=None, demean=True):
    """Estimate AR(p) parameters from a sequence X using Yule-Walker equation.

    Parameters
    ----------
    X : array-like
        1d array
    aic: bool
        If ``True``, then the Akaike Information Criterion is used to choose
        the order of the autoregressive model. If ``False``, the model of order
        ``order.max`` is fitted.
    order_max : integer, optional
        Maximum order of model to fit. Defaults to the smaller of N-1 and
        10*log10(N) where N is the length of the sequence.
    demean : bool
        True, the mean is subtracted from `X` before estimation.

    Returns
    -------
    rho : array, shape=(order,)
        The autoregressive coefficients
    sigma2 : float
        Variance of the nosie term
    aic : float
        Akaike Information Criterion
    """
    # this code is adapted from https://github.com/statsmodels/statsmodels.
    # changes are made to increase compability with R's ``ar.yw``.
    X = np.array(X)
    if demean:
        X -= X.mean()
    n = X.shape[0]

    if X.ndim > 1 and X.shape[1] != 1:
        raise ValueError("expecting a vector to estimate AR parameters")

    if order_max is None:
        order_max = min(n - 1, int(10 * np.log10(n)))

    r = np.zeros(order_max+1, np.float64)
    r[0] = (X**2).sum() / n

    for k in range(1, order_max+1):
        r[k] = (X[0:-k]*X[k:]).sum() / n

    orders = np.arange(1, order_max+1) if aic else [order_max]
    aics = np.zeros(len(orders))
    sigmasqs = np.zeros(len(orders))
    rhos = [None for i in orders]

    for i, order in enumerate(orders):
        r_left = r[:order]
        r_right = r[1:order+1]

        # R = toeplitz(r[:-1])
        R = toeplitz(r_left)
        # rho = np.linalg.solve(R, r[1:])
        rho = np.linalg.solve(R, r_right)
        # sigmasq = r[0] - (r[1:]*rho).sum()
        sigmasq = r[0] - (r_right*rho).sum()
        aic = len(X) * (np.log(sigmasq) + 1) + 2*order + 2*demean
        # R compability
        sigmasq = sigmasq * len(X)/(len(X) - (order + 1))

        aics[i] = aic
        sigmasqs[i] = sigmasq
        rhos[i] = rho

    index = np.argmin(aics)
    return rhos[index], sigmasqs[index]

## Solution to the Bayesian approach: (Approach 1)

Feel free to change the cell structure below ....

### Task 1

The posterior distribution density, up to an unknown normalising constant is such that:

$$
\begin{array}{rl}
\pi(\xi_1,\xi_2,\xi_3,\xi_4,\xi_5,\sigma_y^2|y) \propto& \mathcal{L}(y|\xi,\sigma_y^2) \pi(\xi_1, \xi_2, \xi_3, \xi_4, \xi_5, \sigma^2) \\
\ \propto & ...
\end{array}
$$

### Task 2 (initialise the code)

In [7]:
# =====================
# LAOD REQUIRED MODULES
# =====================

%matplotlib inline
from IPython.core.pylabtools import figsize
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as stats
import pymc as pm
import pandas as pd
from ipywidgets import interactive
import scipy as scipy 

In [ ]:
# ==================================
# OTEHR AUXILIARY FUNCTIONS REQUIRED
# ==================================

# ...

In [8]:
# ================================
# COMPUTER MODEL RELATED FUNCTIONS
# ================================

# ...

In [9]:
# ============================
# DEFINE THE POSTERIOR DENSITY
# ============================

# Define the posterior distribution density, 
#  up to a common normalising constant, 
#  and in the log scale (for computational stability)

# Data
import pandas as pd
catalysis_data = pd.read_csv('../data/catalysis.csv', index_col=0)
thedata = np.array(catalysis_data.values[6])

# Log Posterior PDF up to an unknown normalising constant

# ...

### Task 2

In [10]:
# ================
# THE MCMC SAMPLER
# ================

# ...

### Task 3

In [11]:
# ========================
# SAMPLE EXAMINATION
# ========================

# the average acceptance probebilities ...
# the scale parameter of the proposals if Random Walk Matropolis was used ...
# the integrated autocorrelation time for the generated sample ...

#....

In [12]:
# ========================
# Trace plots of the sample
# ========================

#....

In [13]:
# ========================
# Autocorrelation plots
# ========================

#....

### Task 4

In [14]:
# ================================================
# TRANSFORMATION THE SAMPLE FROM $\xi$ to $\kappa$
# ================================================

#....

In [15]:
# ================================================
# Posterior expected values with standard errors
# ================================================

#....

In [16]:
# ================================================
# Marginal posterior densities of $\kappa$, and $\sigma_y^2$
# ================================================

#....

In [17]:
# ================================================
# Marginal posterior densities (Part 2, and pairwise scaterplots)
# ================================================

#....


# HINT:
# let's say the sample is stored in matrix mat 
# them
# 
# from pandas.tools.plotting import scatter_matrix
# from pandas import DataFrame
# df = DataFrame(data=mat, columns=['k_1', 'k_2', 'k_3', 'k_4', 'k_5']) ;

# References

<mark> <b>The bib file biblio.bib was not found

</b> </mark>(<a id="cit-katsounaros" href="#call-katsounaros">?</a>) !! _This reference was not found in biblio.bib _ !!

